In [2]:
import tensorflow as tf 
from tensorflow import keras
import numpy as np
from tensorflow.keras import layers,Sequential
from random import randint

In [3]:
def actual_function_analytic(x):
    return x*x+5

In [4]:
x_train=[]
y_train=[]
X=-10
while X<10:
    x_train.append(X)
    y_train.append(actual_function_analytic(X))
    X+=0.01

index = [i for i in range(len(y_train))]
tf.random.shuffle(index)
for i in range(len(y_train)):
    x_train[i] = x_train[index[i]]
    y_train[i] = y_train[index[i]]
    
x_test=[]
y_test=[]
X=10
while X<11:
    x_test.append(X)
    y_test.append(actual_function_analytic(X))
    X+=0.1
X=-11
while X<-10:
    x_test.append(X)
    y_test.append(actual_function_analytic(X))
    X+=0.1
index = [i for i in range(len(y_test))]
tf.random.shuffle(index)
for i in range(len(y_test)):
    x_test[i] = x_test[index[i]]
    y_test[i] = y_test[index[i]]
    
(x_eval,y_eval)=(x_train[:len(x_train)//20],y_train[:len(x_train)//20])
(x_train,y_train)=(x_train[len(x_train)//20:],y_train[len(x_train)//20:])

x_train=tf.convert_to_tensor(x_train,dtype=tf.float32)
y_train=tf.convert_to_tensor(y_train,dtype=tf.float32)
x_test=tf.convert_to_tensor(x_test,dtype=tf.float32)
y_test=tf.convert_to_tensor(y_test,dtype=tf.float32)
x_eval=tf.convert_to_tensor(x_eval,dtype=tf.float32)
y_eval=tf.convert_to_tensor(y_eval,dtype=tf.float32)

In [5]:
def forward(x,w1,b1,w2,b2):
    #隐藏层前向计算
    h1=x@w1+b1
    h1=tf.nn.relu(h1)
    #输出层前向计算
    h2=h1@w2+b2
    return h2

In [6]:
def initialize_group():
    chrome=tf.random.normal([NG,C_length])
    return chrome

In [7]:
def decode_to_param(chrome):
    #隐藏层张量
    w1=tf.reshape(chrome[:5],[1,5])
    b1=tf.reshape(chrome[5:10],[5])
    #输出层张量
    w2=tf.reshape(chrome[10:15],[5,1])
    b2=tf.reshape(chrome[-1],[1])
    return w1,b1,w2,b2

In [8]:
def F_fitness(w1,b1,w2,b2):
    batch_size=20
    index=[i for i in range(y_train.get_shape()[0])]
    index=tf.random.shuffle(index)
    err=0
    for i in range(batch_size):
        x=tf.reshape(x_train[index[i]],[1,1])
        o=forward(x,w1,b1,w2,b2)
        #err+=pow(o-y_train[index[i]],2)
        err+=tf.losses.MSE(o,y_train[index[i]])
    err/=batch_size
    return 1/err

In [8]:
def evaluate_group(group):
    score=[]
    for chrome in group:
        w1,b1,w2,b2=decode_to_param(chrome)
        s=F_fitness(w1,b1,w2,b2)
        score.append(tf.reshape(s,[1]))
    return score

In [9]:
def roulette_pick(group,s):
    p=tf.random.uniform([1])
    for i in range(len(s)):
        if p<s[i]:return group[i]
    return group[len(s)-1]

In [10]:
def select(group,score):
    new_group=[]
    norm_score=[score[i]/sum(score) for i in range(len(score))]
    norm_score_cum=np.cumsum(norm_score)
    for i in range(len(group)):
        new_group.append(roulette_pick(group,norm_score_cum))
    return new_group

In [11]:
def crossover(group):
    for i in range(0,len(group),2):
        p=tf.random.uniform([1])
        if p<PC:
            place=randint(0,C_length-1)
            group[i],group[i+1]=tf.concat([group[i][:place],group[i+1][place:]],axis=0),\
                                tf.concat([group[i+1][:place],group[i][place:]],axis=0)

In [12]:
def mutation(group):
    for i in range(len(group)):
        p=tf.random.uniform([1])
        if p<PM:
            place=randint(0,C_length-1)
            group[i]=tf.concat([group[i][:place],
                              group[i][place]+tf.random.uniform([1],minval=-5,maxval=5),
                                group[i][place+1:]],
                             axis=0)

In [13]:
def elitist(group,score,elit,elit_score):
    index=score.index(max(score))
    if score[index]>elit_score: return group[index],score[index]
    else:                       return elit,elit_score

In [14]:
def elit_replace(group,score,elit,elit_score):
    index=score.index(min(score))
    group[index],score[index]=elit,elit_score

In [15]:
#遗传算法全局变量定义区
PM=0.01
PC=0.1
NG=20
Generate=50
C_length=16

In [16]:
def GA():
    group=initialize_group()
    score=evaluate_group(group)
    elit_score=0
    elit=0
    for i in range(Generate):
        elit,elit_score=elitist(group,score,elit,elit_score)
        group=select(group,score)#Group是一个张量列表
        crossover(group)
        mutation(group)
        score=evaluate_group(group)
        elit_replace(group,score,elit,elit_score)
        print(max(score),sum(score)/len(score))
GA()

NameError: name 'np' is not defined